### 사전 준비 사항 

#### (1) uv add (터미널)

```bash
uv add pdfplumber sentence-transformers faiss-cpu numpy torch python-dotenv openai
```

#### (2) .env 파일 세팅
```bash
OPENAI_API_KEY = ""
HF_TOKEN = ""
```

In [1]:
import pdfplumber
import os
from pathlib import Path
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

/Users/won/dev/00_codeit/0_mission/200_DL_RAG/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()
MODEL = "gpt-4o-mini"

In [3]:
import json

BASE_DIR = Path.cwd().parent  # /codeit-part3-team4
RAW_FOLDER = BASE_DIR / "data/raw/files"  # 수정된 경로

# 폴더에서 PDF 목록 가져오기
def get_pdf_paths(folder_path: Path | str) -> list[Path]:
    folder = Path(folder_path)
    pdf_paths = [p for p in folder.glob("*.pdf")]
    return sorted(pdf_paths)

PDF_PATH_LIST = get_pdf_paths(RAW_FOLDER)


all_data_path = os.path.join(BASE_DIR, "data", "ALL_DATA.json")

with open(all_data_path, "r", encoding="utf-8") as f:
    ALL_DATA = json.load(f)

In [4]:
# PDF to text
def extract_text(pdf_path: Path | str) -> str:
    texts = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            texts.append(page.extract_text() or "")
    return "\n".join(texts)

In [5]:
# Chunking
def chunk(text: str, size: int = 800) -> list[str]:
    return [text[i:i+size] for i in range(0, len(text), size)]

In [6]:
# 임베딩 및 인덱스 만들기
def build_index(chunks: list[str]) -> tuple:
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embs = model.encode(chunks)

    index = faiss.IndexFlatL2(embs.shape[1])
    index.add(np.array(embs).astype("float32"))
    return index, model, chunks

In [7]:
# RFP 분석용 프롬프트
RFP_PROMPT = """
너는 정부·공공기관 제안요청서(RFP)를 분석하는 전문가다.
아래 컨텍스트는 하나의 정부 RFP 문서에서 추출된 내용이다.

다음 기준에 따라 **핵심 정보만 구조적으로 요약**해라.

[요약 규칙]
- 추측하지 말고, 문서에 명시된 내용만 사용
- 불필요한 설명, 배경, 미사여구 제거
- 사업 참여 판단에 필요한 정보 위주로 요약
- 항목이 없으면 "명시 없음"으로 표기

[출력 형식]
아래 각 항목은 **질문에 답하는 형태**로 작성하되,
반드시 `key: 답변` 형식으로 출력하라.
(예시) project_name: 고려대학교 차세대 포털·학사 정보시스템 구축 사업

project_name: 사업(용역)명은 무엇인가?
agency: 발주 기관(수요기관)은 어디인가?
purpose: 사업 목적(추진 배경)은 무엇인가?
budget: 총 사업 예산(사업비)은 얼마인가?
base_amount: 추정 가격/기초금액은 얼마인가?
contract_type: 계약 방식(일반경쟁/제한경쟁/협상에 의한 계약 등)은 무엇인가?
deadline: 입찰/제안서 제출 마감일시는 언제인가?
submission_method: 제출 장소 또는 제출 방법(온라인/방문/우편)은 무엇인가?
submission_docs: 제출 서류 목록은 무엇인가?
format_requirement: 제안서(기술/가격) 제출 파일 형식이나 양식은 무엇인가?
duration: 사업 수행 기간은 얼마나 되는가?
start_date: 사업 착수(시작) 예정일은 언제인가?
end_date: 사업 종료(납품) 예정일은 언제인가?
scope: 제안 요청 범위(Scope/업무 범위)는 무엇인가?
requirements_must: 필수 요구사항(기능/성능/보안 등)은 무엇인가?
requirements_optional: 선택 요구사항(우대/가점)에는 무엇이 있는가?
eval_items: 평가 항목(기술/가격 등) 구성은 어떻게 되는가?
eval_weights: 평가 배점(점수) 비율은 어떻게 되는가?
price_eval: 가격 평가 방식(최저가/협상 등)은 무엇인가?
presentation: 제안 발표(PT/발표평가)가 있는가? 있다면 일정은?
qa_period: 질의응답(Q&A) 접수 기간은 언제인가?
qa_method: 질의응답 제출 방법/양식은 무엇인가?
briefing: 현장설명회가 있는가? 있다면 일정/장소는?
eligibility: 참가 자격 요건(면허/실적/인증/등급)은 무엇인가?
consortium: 공동수급/하도급/컨소시엄 가능 여부는?
staffing: 필수 인력 요건(PM/보안담당 등)이 있는가?
security: 보안/개인정보 요구사항(인증, ISMS 등)은 무엇인가?
deliverables: 납품물(산출물) 목록은 무엇인가?
maintenance: 유지보수/운영 조건(기간/범위)은 무엇인가?
contact: 문의처(담당자/전화/이메일)는 무엇인가?

[컨텍스트]
{context}

이제 위 형식에 맞춰 한국어로 요약해라.
"""

In [8]:
# 쿼리 답변
def answer(index, model, chunks, query: str, top_k: int = 15) -> str:
    q_emb = model.encode([query])
    _, I = index.search(np.array(q_emb).astype("float32"), top_k)
    
    context = "\n\n".join(chunks[i] for i in I[0])
    prompt = RFP_PROMPT.format(context=context)
    
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "RFP 전문 분석가. 형식 엄수."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=2000,
    )
    return resp.choices[0].message.content

In [9]:
# 데이터 폴더 설정
docs = get_pdf_paths(RAW_FOLDER)
print(f"발견된 PDF: {len(docs)}개")
for i, doc in enumerate(docs):
    print(f"{i}: {doc.name}")

발견된 PDF: 100개
0: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf
1: (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf
2: (사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf
3: (재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf
4: 2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf
5: BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).pdf
6: KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .pdf
7: 경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.pdf
8: 경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.pdf
9: 경기도사회서비스원_2024년 통합사회정보시스템 운영지원.pdf
10: 경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).pdf
11: 경희대학교_[입찰공고] 산학협력단 정보시스

In [25]:
def format_table(table: dict) -> str:
    """table_content를 읽기 좋은 텍스트로 변환."""
    tc = table.get("table_content", {})
    cols = tc.get("columns", [])
    rows = tc.get("data", [])
    if not cols and not rows:
        return ""
    
    lines = []
    title = table.get("table_title", "")
    if title:
        lines.append(f"[표] {title}")
    lines.append(" | ".join(str(c or "") for c in cols))
    for row in rows:
        lines.append(" | ".join(str(c or "") for c in row))
    return "\n".join(lines)


def extract_text_from_alldata(doc_name: str, all_data: dict) -> str | None:
    """ALL_DATA.json에서 메타데이터를 최대한 활용하여 텍스트 추출."""
    if doc_name not in all_data:
        return None
    
    pages = all_data[doc_name]["metadata"]
    texts = []
    
    for page in pages:
        parts = []
        
        # 1) 섹션 헤더 (문맥 경계 표시)
        section = page.get("section")
        if section:
            if isinstance(section, list):
                sec_str = " > ".join(section)
            else:
                sec_str = section
            parts.append(f"[섹션: {sec_str}] (p.{page['page'] + 1})")
        else:
            parts.append(f"(p.{page['page'] + 1})")
        
        # 2) 본문 텍스트
        if page.get("text"):
            parts.append(page["text"])
        
        # 3) 테이블
        if page.get("table"):
            for t in page["table"]:
                table_text = format_table(t)
                if table_text:
                    parts.append(table_text)
        
        texts.append("\n".join(parts))
    
    return "\n\n".join(texts)


In [ ]:
# 전체 문서별 인덱스 저장
doc_indexes = {}

for doc_path in docs:
    print(f"처리 중: {doc_path.name}")

    text = extract_text_from_alldata(doc_path.name, ALL_DATA) or extract_text(doc_path)
    chunks = chunk(text)
    index, model, chunks = build_index(chunks)
    doc_indexes[doc_path] = (index, model, chunks)

    break
print("모든 문서 인덱싱 완료!")

처리 중: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2140.93it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


처리 중: (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1796.49it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


처리 중: (사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf


...

처리 중: 서울시립대학교_[사전공개] 학업성취도 다차원 종단분석 통합시스템 1차.pdf


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2022.54it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


처리 중: 서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2086.50it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


모든 문서 인덱싱 완료!


In [ ]:
# 특정 문서의 청크 몇 개를 뽑아서 확인
test_name = list(ALL_DATA.keys())[0]
text = extract_text_from_alldata(test_name, ALL_DATA)
chunks_test = chunk(text)

print(f"문서: {test_name}")
print(f"총 청크 수: {len(chunks_test)}")
print()

# 섹션/페이지/테이블이 포함된 청크 찾기
for i, c in enumerate(chunks_test):
    has_section = "[섹션:" in c
    has_page = "(p." in c
    has_table = "[표]" in c
    if has_section or has_table:
        print(f"=== 청크 {i} ({len(c)}자) ===")
        print(c[:500])
        print("...\n")
        if i > 5:
            break

문서: 사단법인 보험개발원_실손보험 청구 전산화 시스템 구축 사업.pdf
총 청크 수: 235

=== 청크 3 (800자) ===
················································· 102     [붙임1] 외주용역사업 보안특약················································· 103  [붙임2] 제출서식··········································································· 106

[섹션: 1. 개요 > 2. 추진배경 및 필요성·] (p.4)
- 2 -
Ⅰ  사업 개요1. 개요 □ 사 업 명 : 실손보험 청구 전산화 시스템 구축 □ 사업기간 : 계약일로부터 19개월    ※ 1차: ’24년 4월~10월 25일, 2차: ’24년 10월~’25년 10월 25일 □ 계약방식 : 일반경쟁입찰, 협상에 의한 계약  ※ 본 제안요청서에서는 사업예산이 포함되지 않음2. 추진배경 및 필요성 □ 실손보험은 국민건강보험의 보완형으로 도입되어 ’23년말 국민 약 4천만명이 가입한 일상생활에 
...

=== 청크 4 (800자) ===
보험업법을 개정* *’23.10.24 공포 → ’24.10.25 병원급 시행(1단계) → ’25.10.25 의원 및 약국 시행(2단계)  o 이번 개정 법률은 요양기관과 보험사 간 전송대행기관을 두고 청구

[섹션: 3. 실손보험 청구 전산화 시스템 개요] (p.5)
- 3 -
전산화를 구현하는 것으로 별도 시스템 구축을 필요로 하고 있음3. 실손보험 청구 전산화 시스템 개요 □ 참여기관 : 전국 요양기관(병의원 및 약국), 실손보험계약을 보유 또는 보유예정인 보험사
<그림1>실손보험 청구 전산화 시스템 개념도 □ 주요기능   o 진료를 받은 보험가입자의 요청에 따라 요양기관에서 보험사로 보험금 청구 필요서류를 전자적 방법으로 전송   

In [31]:
test_doc = docs[0]

index, model, chunks = doc_indexes[test_doc]

print(f"총 청크 수: {len(chunks)}")
print()
print("=== 청크 0 ===")
print(chunks[0][:500])
print()
print("=== 청크 1 ===")
print(chunks[1][:500])


총 청크 수: 255

=== 청크 0 ===
(p.1)
202420242024202420242024202420242024202420242024202420242024202420242024년 년 년 년 년 년 년 년 년 년 년 년 년 년 년 년 년 년 ｢｢｢｢｢｢｢｢｢｢｢｢｢｢｢｢｢｢벤처확인종합관리시스템 벤처확인종합관리시스템 벤처확인종합관리시스템 벤처확인종합관리시스템 벤처확인종합관리시스템 벤처확인종합관리시스템 벤처확인종합관리시스템 벤처확인종합관리시스템 벤처확인종합관리시스템 벤처확인종합관리시스템 벤처확인종합관리시스템 벤처확인종합관리시스템 벤처확인종합관리시스템 벤처확인종합관리시스템 벤처확인종합관리시스템 벤처확인종합관리시스템 벤처확인종합관리시스템 벤처확인종합관리시스템 기능 기능 기능 기능 기능 기능 기능 기능 기능 기능 기능 기능 기능 기능 기능 기능 기능 기능 고도화고도화고도화고도화고도화고도화고도화고도화고도화고도화고도화고도화고도화고도화고도화고도화고도화고도화｣｣｣｣｣｣｣｣｣｣｣｣｣｣｣｣｣｣2024년 ｢벤처확인종합관리시스템 기능 고도화｣

=== 청크 1 ===
············· 3    2. 추진방안··························································································· 5    3. 추진내용··························································································· 9    4. 제안요청내용················································································· 24    5. 입찰관련사항················································································· 78    6. 제안서작성요령····························

In [ ]:
# 테스트 1: 단일 문서 (docs[0])
if docs:
    test_doc = docs[0]
    print(f"\n=== {test_doc.name} 분석 ===")
    index, model, chunks = doc_indexes[test_doc]
    result = answer(index, model, chunks, "RFP 분석해줘")
    print(result)

NameError: name 'docs' is not defined

In [ ]:
# # 테스트 2: 전체 문서 루프 (비용 주의! 주석 해제 후 사용)
# # 실행 완료시 rfp_summaries.json 파일이 생성됩니다.

# results = {}
# for doc_path in docs:
#     print(f"\n=== {doc_path.name} 분석 ===")
#     index, model, chunks = doc_indexes[doc_path]
#     result = answer(index, model, chunks, "RFP 분석해줘")
#     results[doc_path.name] = result
#     print(result)

# # JSON 저장
# import json
# with open(BASE_DIR / "rfp_summaries.json", "w", encoding="utf-8") as f:
#     json.dump(results, f, ensure_ascii=False, indent=2)
# print("전체 분석 완료!")


=== 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf 분석 ===
project_name: 고려대학교 차세대 포털·학사 정보시스템 구축 사업  
agency: 고려대학교  
purpose: 학령인구 감소 및 교육환경 변화에 대응하고, 분산된 시스템 통합 및 데이터 기반 대학경영 지원을 개선하기 위함  
budget: 11,270,000,000원 (V.A.T 포함, 3년 분할 지급)  
base_amount: 명시 없음  
contract_type: 제한 경쟁 입찰(협상에 의한 계약)  
deadline: 명시 없음  
submission_method: 명시 없음  
submission_docs: 명시 없음  
format_requirement: 명시 없음  
duration: 계약일로부터 24개월 이내  
start_date: 명시 없음  
end_date: 사업 종료일로부터 12개월의 무상유지보수 기간 포함  
scope: 포털 및 학사 정보시스템 관련 구축 및 통합  
requirements_must: 학적 기본관리, 학사일정관리 등 명시된 기능 요구 사항  
requirements_optional: 명시 없음  
eval_items: 기술 평가, 가격 평가  
eval_weights: 명시 없음  
price_eval: 협상에 의한 가격 평가  
presentation: 명시 없음  
qa_period: 명시 없음  
qa_method: 명시 없음  
briefing: 명시 없음  
eligibility: 명시 없음  
consortium: 명시 없음  
staffing: 명시 없음  
security: 명시 없음  
deliverables: 프로그램 목록, 프로그램명세서, 화면정의서, 사용자매뉴얼 등  
maintenance: 하자보수 계획 포함, 사업 종료 후 무상유지보수 12개월  
contact: 명시 없음  

=== 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf 분석 ===
project_name: 2025년